<h1 style="color:#65AE11;">Copy Compute Overlap with Multiple GPUs</h1>

In this section you will learn the indexing strategies required to combine the techniques you have learned so far and apply copy/compute overlap when using multiple GPUs.

<h2 style="color:#65AE11;">Objectives</h2>

By the time you complete this section you will understand:

* How streams are associated with each GPU device
* How to create non-default streams for multiple GPUs
* How to perform copy/compute overlap on multiple GPUs

<h2 style="color:#65AE11;">Streams and Multiple GPUs</h2>

Each GPU has its own default stream. Non-default streams can be created, utilized, and destroyed for the currently active GPU device. Care must be taken not to launch kernels in streams not associated with the currently active GPU.

<h2 style="color:#65AE11;">Creating Multiple Streams for Multiple GPUs</h2>

When using multiple non-default streams on multiple GPUs, rather than simply store streams in an array as we did previously, we will store them in a 2D array, with each row containing the streams for a single GPU:

```c
cudaStream_t streams[num_gpus][num_streams]; // 2D array containing number of streams for each GPU.

// For each available GPU...
for (uint64_t gpu = 0; gpu < num_gpus; gpu++) {
    // ...set as active device...
    cudaSetDevice(gpu);
    for (uint64_t stream = 0; stream < num_streams; stream++)
        // ...create and store its number of streams.
        cudaStreamCreate(&streams[gpu][stream]);
}
```

<h2 style="color:#65AE11;">Data Chunk Sizes for Multiple Streams on Multiple GPUs</h2>

Indexing into global data becomes even more tricky when using multiple non-default streams with multiple GPUs. It can be helpful to define data chunk sizes for a single stream, as well as data chunk sizes for an entire GPU. Here we will continue to use the robust indexing strategies discussed in [Copy Compute Considerations](../08_Copy_Compute_Considerations/Copy_Compute_Considerations.ipynb):

```c
// Each stream needs num_entries/num_gpus/num_streams data. We use round up division for
// reasons previously discussed.
const uint64_t stream_chunk_size = sdiv(sdiv(num_entries, num_gpus), num_streams);

// It will be helpful to also to have handy the chunk size for an entire GPU.
const uint64_t gpu_chunk_size = stream_chunk_size*num_streams;
```

<h2 style="color:#65AE11;">Allocating Memory with Multiple Streams for Multiple GPUs</h2>

GPU memory is not allocated with streams, so allocation here looks similar to our previous work with multiple GPUs, only care needs to be taken to use a chunk size for the entire GPU, and not one of its streams:

```c
// For each GPU...
for (uint64_t gpu = 0; gpu < num_gpus; gpu++) {

    // ...set device as active...
    cudaSetDevice(gpu);

    // ...use a GPU chunk's worth of data to calculate indices and width...
    const uint64_t lower = gpu_chunk_size*gpu;
    const uint64_t upper = min(lower+gpu_chunk_size, num_entries);
    const uint64_t width = upper-lower;

    // ...allocate data.
    cudaMalloc(&data_gpu[gpu], sizeof(uint64_t)*width);
}
```

<h2 style="color:#65AE11;">Copy/Compute Overlap with Multiple Streams for Multiple GPUs</h2>

For each GPU, we will perform copy/compute overlap in multiple non-default streams. This technique is very similar as that with only one GPU, only we must do it while looping over each GPU, and, take some additional care with indexing into the data. Work through this section slowly:

```c
// For each GPU...
for (uint64_t gpu = 0; gpu < num_gpus; gpu++) {
    // ...set device as active.
    cudaSetDevice(gpu);
    // For each stream (on each GPU)...
    for (uint64_t stream = 0; stream < num_streams; stream++) {

        // Calculate index offset for this stream's chunk of data within the GPU's chunk of data...
        const uint64_t stream_offset = stream_chunk_size*stream;
        
        // ...get the lower index within all data, and width of this stream's data chunk...
        const uint64_t lower = gpu_chunk_size*gpu+stream_offset;
        const uint64_t upper = min(lower+stream_chunk_size, num_entries);
        const uint64_t width = upper-lower;

        // ...perform async HtoD memory copy...
        cudaMemcpyAsync(data_gpu[gpu]+stream_offset, // This stream's data within this GPU's data.
                        data_cpu+lower,              // This stream's data within all CPU data.
                        sizeof(uint64_t)*width,      // This stream's chunk size worth of data.
                        cudaMemcpyHostToDevice,
                        streams[gpu][stream]);       // Using this stream for this GPU.

        kernel<<<grid, block, 0, streams[gpu][stream]>>>    // Using this stream for this GPU.
            (data_gpu[gpu]+stream_offset,                   // This stream's data within this GPU's data.
             width);                                        // This stream's chunk size worth of data.

        cudaMemcpyAsync(data_cpu+lower,              // This stream's data within all CPU data.
                        data_gpu[gpu]+stream_offset, // This stream's data within this GPU's data.
                        sizeof(uint64_t)*width,
                        cudaMemcpyDeviceToHost,
                        streams[gpu][stream]);       // Using this stream for this GPU.
    }
}
```

<h2 style="color:#65AE11;">Check for Understanding</h2>

Please answer the following to confirm you've learned the main objectives of this section. You can display the answers for each question by clicking on the "..." cells below the questions.

---

**Which of the following is true? Choose all that apply.**

1. The default stream is reserved for the GPU device with index `0`
2. Each GPU has its own default stream
3. Non-default streams can be created for the currently active GPU
4. A single non-default stream can be used to perform operations on multiple GPUs

**Answer: 2, 3**

---

**Kernels can be launched on a not-currently-active GPU by launching it in a stream associated with a not-currently-active GPU.**

1. True
2. False

**Answer: 2**

Kernel launches will fail if issued into a stream not associated with the currently active GPU device.

---

<h2 style="color:#65AE11;">Next</h2>

You now have all the techniques at your disposal needed to perform copy/compute overlap on multiple GPUs. In the next section you will once again refactor the cipher application, this time for tremendous speedups by performing copy/compute overlap while doing computations on multiple GPUs.

Please continue to the next section: [*Exercise: MGPU Streams*](../13_Exercise_MGPU_Streams/Exercise_MGPU_Streams.ipynb).